# 52. CNN - Models

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. AlexNet을 구현해본다. 
2. VGGNet을 구현해본다. 
3. GoogLeNet을 구현해본다. 
4. ResNet을 구현해본다. 

#### 사용데이터.
1. The CIFAR-10 dataset : https://www.cs.toronto.edu/~kriz/cifar.html

![image](https://github.com/blackdew/dplus-tensorflow/blob/master/img/06_network_flowchart.png?raw=true)

In [0]:
# gpu 사용확인 

import tensorflow as tf
tf.test.gpu_device_name()


In [0]:
import tensorflow as tf
import numpy as np
import random

# cifar10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()

train_y = train_y.reshape(-1,)
test_y = test_y.reshape(-1,)

'''
트레이닝 데이터로 활용할 50000개 이미지, 테스트용 10000개 이미지
32 * 32 * 3 사이즈의 이미지가 어레이에 담겨있다.
레이블은 one-hot encoding이 되어 있지 않다.
'''

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# 이미지 출력을 위한 함수
def cifar10_plot(i):
    pixels = test_x[i]
    a = int(test_y[i])
    plt.title('Label: {} ({})'.format(class_names[a], a))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

In [0]:
cifar10_plot(np.random.randint(0, high=9999))


## CIFAR-10 + AlexNet

![image](http://mblogthumb1.phinf.naver.net/20160314_204/laonple_14579300603930fQ7q_PNG/%C0%CC%B9%CC%C1%F6_2.png?type=w2)


In [0]:
import tensorflow as tf
import numpy as np
import random

# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders
lr = tf.placeholder(tf.float32) # learning rate
kp = tf.placeholder(tf.float32) # dropout

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


In [0]:
#########################
# ConvLayer 01-1 / (11, 11, 3, 48) but, (3, 3, 3, 8)

# for Conv Layer 01 filter 
w11 = tf.Variable(tf.random_normal([3, 3, 3, 8], stddev=0.01))
b11 = tf.Variable(tf.random_normal([8], stddev=0.01))
tf.add_to_collection('weights', w11)

# Convolution Layer -> (?, 32, 32, 8)
conv11 = tf.add(tf.nn.conv2d(X, w11, strides=[1, 1, 1, 1], padding='SAME'), b11)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv11, [0])
conv11 = tf.nn.batch_normalization(conv11, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv11 = tf.nn.relu(conv11)

# Pooling Layer -> (?, 16, 16, 8)
pool11 = tf.nn.max_pool(conv11, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#########################
# ConvLayer 01-2 / (5, 5, 48, 128) but, (3, 3, 8, 16)

# for Conv Layer 02 filter
w12 = tf.Variable(tf.random_normal([3, 3, 8, 16], stddev=0.01))
b12 = tf.Variable(tf.random_normal([16], stddev=0.01))
tf.add_to_collection('weights', w12)

# Convolution Layer -> (?, 16, 16, 16)
conv12 = tf.add(tf.nn.conv2d(pool11, w12, strides=[1, 1, 1, 1], padding='SAME'), b12)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv12, [0])
conv12 = tf.nn.batch_normalization(conv12, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv12 = tf.nn.relu(conv12)

# Pooling Layer -> (?, 8, 8, 16)
pool12 = tf.nn.max_pool(conv12, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
#########################
# ConvLayer 02-1 / (11, 11, 3, 48) but, (3, 3, 3, 8)

# for Conv Layer 01 filter
w21 = tf.Variable(tf.random_normal([3, 3, 3, 8], stddev=0.01))
b21 = tf.Variable(tf.random_normal([8], stddev=0.01))
tf.add_to_collection('weights', w21)

# Convolution Layer -> (?, 32, 32, 8)
conv21 = tf.add(tf.nn.conv2d(X, w21, strides=[1, 1, 1, 1], padding='SAME'), b21)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv21, [0])
conv21 = tf.nn.batch_normalization(conv21, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv21 = tf.nn.relu(conv21)

# Pooling Layer -> (?, 16, 16, 8)
pool21 = tf.nn.max_pool(conv21, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#########################
# ConvLayer 02-2 / (5, 5, 48, 128) but, (3, 3, 8, 16)

# for Conv Layer 02 filter
w22 = tf.Variable(tf.random_normal([3, 3, 8, 16], stddev=0.01))
b22 = tf.Variable(tf.random_normal([16], stddev=0.01))
tf.add_to_collection('weights', w22)

# Convolution Layer -> (?, 16, 16, 16)
conv22 = tf.add(tf.nn.conv2d(pool21, w22, strides=[1, 1, 1, 1], padding='SAME'), b22)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv22, [0])
conv22 = tf.nn.batch_normalization(conv22, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv22 = tf.nn.relu(conv22)

# Pooling Layer -> (?, 8, 8, 16)
pool22 = tf.nn.max_pool(conv22, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



In [0]:
#########################
# ConvLayer 01-3
# p21 * (3, 3, 128, 192) + p22 * (3, 3, 128, 192). but
# p21 * (3, 3, 16, 24) + p22 * (3, 3, 16, 24)

# for Conv Layer 03 filter
w131 = tf.Variable(tf.random_normal([3, 3, 16, 24], stddev=0.01))
b131 = tf.Variable(tf.random_normal([24], stddev=0.01))
tf.add_to_collection('weights', w131)

# Convolution Layer -> (?, 8, 8, 24)
conv131 = tf.add(tf.nn.conv2d(pool12, w131, strides=[1, 1, 1, 1], padding='SAME'), b131)

# for Conv Layer 03 filter
w132 = tf.Variable(tf.random_normal([3, 3, 16, 24], stddev=0.01))
b132 = tf.Variable(tf.random_normal([24], stddev=0.01))
tf.add_to_collection('weights', w132)

# Convolution Layer -> (?, 8, 8, 24)
conv132 = tf.add(tf.nn.conv2d(pool22, w132, strides=[1, 1, 1, 1], padding='SAME'), b132)

# convolution result merge -> (?, 8, 8, 24)
conv13 = tf.add(conv131, conv132)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv13, [0])
conv13 = tf.nn.batch_normalization(conv13, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv13 = tf.nn.relu(conv13)


#########################
# ConvLayer 01-4 / (3, 3, 192, 192), but (3, 3, 24, 24)

# for Conv Layer 04 filter
w14 = tf.Variable(tf.random_normal([3, 3, 24, 24], stddev=0.01))
b14 = tf.Variable(tf.random_normal([24], stddev=0.01))
tf.add_to_collection('weights', w14)

# Convolution Layer -> (?, 8, 8, 24)
conv14 = tf.add(tf.nn.conv2d(conv13, w14, strides=[1, 1, 1, 1], padding='SAME'), b14)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv14, [0])
conv14 = tf.nn.batch_normalization(conv14, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv14 = tf.nn.relu(conv14)
    
    
#########################
# ConvLayer 01-5 / (3, 3, 192, 128), but (3, 3, 24, 16)

# for Conv Layer 05 filter
w15 = tf.Variable(tf.random_normal([3, 3, 24, 16], stddev=0.01))
b15 = tf.Variable(tf.random_normal([16], stddev=0.01))
tf.add_to_collection('weights', w15)

# Convolution Layer -> (?, 8, 8, 16)
conv15 = tf.add(tf.nn.conv2d(conv14, w15, strides=[1, 1, 1, 1], padding='SAME'), b15)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv15, [0])
conv15 = tf.nn.batch_normalization(conv15, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv15 = tf.nn.relu(conv15)

# Pooling Layer -> (?, 4, 4, 16)
pool15 = tf.nn.max_pool(conv15, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
#########################
# ConvLayer 02-3
# p21 * (3, 3, 128, 192) + p22 * (3, 3, 128, 192). but
# p21 * (3, 3, 16, 24) + p22 * (3, 3, 16, 24)

# for Conv Layer 03 filter
w231 = tf.Variable(tf.random_normal([3, 3, 16, 24], stddev=0.01))
b231 = tf.Variable(tf.random_normal([24], stddev=0.01))
tf.add_to_collection('weights', w231)

# Convolution Layer -> (?, 8, 8, 24)
conv231 = tf.add(tf.nn.conv2d(pool12, w231, strides=[1, 1, 1, 1], padding='SAME'), b231)

# for Conv Layer 03 filter
w232 = tf.Variable(tf.random_normal([3, 3, 16, 24], stddev=0.01))
b232 = tf.Variable(tf.random_normal([24], stddev=0.01))
tf.add_to_collection('weights', w232)

# Convolution Layer -> (?, 8, 8, 24)
conv232 = tf.add(tf.nn.conv2d(pool22, w232, strides=[1, 1, 1, 1], padding='SAME'), b232)

# convolution result merge -> (?, 8, 8, 24)
conv23 = tf.add(conv231, conv232)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv23, [0])
conv23 = tf.nn.batch_normalization(conv23, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv23 = tf.nn.relu(conv23)


#########################
# ConvLayer 02-4 / (3, 3, 192, 192), but (3, 3, 24, 24)

# for Conv Layer 04 filter
w24 = tf.Variable(tf.random_normal([3, 3, 24, 24], stddev=0.01))
b24 = tf.Variable(tf.random_normal([24], stddev=0.01))
tf.add_to_collection('weights', w24)

# Convolution Layer -> (?, 8, 8, 24)
conv24 = tf.add(tf.nn.conv2d(conv23, w24, strides=[1, 1, 1, 1], padding='SAME'), b24)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv24, [0])
conv24 = tf.nn.batch_normalization(conv24, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv24 = tf.nn.relu(conv24)
    
    
#########################
# ConvLayer 01-5 / (3, 3, 192, 128), but (3, 3, 24, 16)

# for Conv Layer 05 filter
w25 = tf.Variable(tf.random_normal([3, 3, 24, 16], stddev=0.01))
b25 = tf.Variable(tf.random_normal([16], stddev=0.01))
tf.add_to_collection('weights', w25)

# Convolution Layer -> (?, 8, 8, 16)
conv25 = tf.add(tf.nn.conv2d(conv24, w25, strides=[1, 1, 1, 1], padding='SAME'), b25)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv25, [0])
conv25 = tf.nn.batch_normalization(conv25, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv25 = tf.nn.relu(conv25)

# Pooling Layer -> (?, 4, 4, 16)
pool25 = tf.nn.max_pool(conv25, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



In [0]:
###########################################
# Flatten layer & Fully Connected Layer 01-1 / (13 * 13 * 256, 4096) but (4 * 4 * 32, 256)

###############################
# Flatten layer
pool5 = tf.concat([pool15, pool25], axis=3)

dim = pool5.get_shape().as_list()
flat_dim = dim[1] * dim[2] * dim[3] # 4 * 4 * 32
flat = tf.reshape(pool5, [-1, flat_dim])

###############################
# Fully Connected Layer 06

# for Final FC 4x4x128 inputs
wfc1 = tf.Variable(tf.random_normal([flat_dim, 256], stddev=0.01))
bfc1 = tf.Variable(tf.random_normal([256]))
tf.add_to_collection('weights', wfc1)

# fc layer
fc1 = tf.add(tf.matmul(flat, wfc1), bfc1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(fc1, [0])
fc1 = tf.nn.batch_normalization(fc1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
fc1 = tf.nn.relu(fc1)
# dropout
fc1 = tf.nn.dropout(fc1, kp)

    
###############################
# Fully Connected Layer 07

# for Final FC 1024 input
wfc2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
bfc2 = tf.Variable(tf.random_normal([256]))
tf.add_to_collection('weights', wfc2)


# fc layer
fc2 = tf.add(tf.matmul(fc1, wfc2), wfc2)
# batch normalization
batch_mean, batch_var = tf.nn.moments(fc2, [0])
fc2 = tf.nn.batch_normalization(fc2, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
fc2 = tf.nn.relu(fc2)
# dropout
fc2 = tf.nn.dropout(fc2, kp)

###############################
# Fully Connected Layer 08

# for Final FC 1000 inputs -> 10 outputs
wfc3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
bfc3 = tf.Variable(tf.random_normal([10]))
tf.add_to_collection('weights', wfc3)

# fc layer
logits = tf.add(tf.matmul(fc2, wfc3), bfc3)

#########################
# Cost & Optimizer

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y), name="cost")

l2_loss = tf.reduce_sum(0.00005 * tf.stack([tf.nn.l2_loss(v) for v in tf.get_collection('weights')])) # LAMBDA = 0.00005 # for weight decay

loss = tf.add(cost, l2_loss)

optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [0]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')

epochs = 20
batch_size = 256
learning_rate = 0.01
dropout = 0.7
n_of_batches = int(len(train_x) / batch_size)

for epoch in range(epochs): 
    print("%dth epoch" % (epoch + 1))

    for i in range(n_of_batches):

        X_batch = train_x[(i * batch_size):((i + 1) * batch_size)]
        Y_batch = train_y[(i * batch_size):((i + 1) * batch_size)]

        sess.run(optimizer, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                       lr: learning_rate, kp: dropout})

        # 학습 상황 디스플레이
        if ((i + 1) % 10 == 0): 
            loss, accuracy = sess.run([cost, acc], 
                                      feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                                 lr: learning_rate, kp: 1.0})
            print("%dth records, training cost: %.3f, accuracy: %.2f" % ((i + 1) * batch_size, loss, accuracy * 100))

print("Training Complete")

In [0]:
###########################
# Evaluation
# accuracy = sess.run(acc, feed_dict={X: test_x, Y: np.eye(10)[test_y], lr: learning_rate, kp: 1.0})
# print('Accuracy: %.2f' % (accuracy * 100))

n_of_batches = int(len(test_x) / batch_size)
accuracy = 0
for i in range(n_of_batches):

    X_batch = test_x[(i * batch_size):((i + 1) * batch_size)]
    Y_batch = test_y[(i * batch_size):((i + 1) * batch_size)]

    accuracy += sess.run(acc, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], lr: learning_rate, kp: 1.0})
    
print('Accuracy: %.2f' % ((accuracy / n_of_batches) * 100))



## CIFAR-10 + VGG16


![image](https://adeshpande3.github.io/assets/VGGNet.png)


In [0]:
import tensorflow as tf
import numpy as np
import random

# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders
lr = tf.placeholder(tf.float32) # learning rate
kp = tf.placeholder(tf.float32) # dropout

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


In [0]:
#########################
# ConvLayer 1, 2 (원본 인풋: 224 * 224 * 3)

# for Conv Layer 01 filter / (3, 3, 3, 64) but (3, 3, 3, 16)
w1 = tf.Variable(tf.random_normal([3, 3, 3, 16], stddev=0.01))
b1 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 32, 32, 16)
conv1 = tf.add(tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME'), b1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv1, [0])
conv1 = tf.nn.batch_normalization(conv1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv1 = tf.nn.relu(conv1)

# for Conv Layer 02 filter / (3, 3, 64, 64) but (3, 3, 16, 16)
w2 = tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=0.01))
b2 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 32, 32, 16)
conv2 = tf.add(tf.nn.conv2d(conv1, w2, strides=[1, 1, 1, 1], padding='SAME'), b2)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv2, [0])
conv2 = tf.nn.batch_normalization(conv2, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv2 = tf.nn.relu(conv2)

# Pooling Layer -> (?, 16, 16, 16)
pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
#########################
# ConvLayer 3, 4 (원래 인풋: 112 * 112 * 64)

# for Conv Layer 03 filter / (3, 3, 64, 128) but (3, 3, 16, 32)
w3 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01))
b3 = tf.Variable(tf.random_normal([32], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 32)
conv3 = tf.add(tf.nn.conv2d(pool2, w3, strides=[1, 1, 1, 1], padding='SAME'), b3)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv3, [0])
conv3 = tf.nn.batch_normalization(conv3, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv3 = tf.nn.relu(conv3)

# for Conv Layer 04 filter / (3, 3, 128, 128) but (3, 3, 32, 32)
w4 = tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=0.01))
b4 = tf.Variable(tf.random_normal([32], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 32)
conv4 = tf.add(tf.nn.conv2d(conv3, w4, strides=[1, 1, 1, 1], padding='SAME'), b4)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv4, [0])
conv4 = tf.nn.batch_normalization(conv4, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv4 = tf.nn.relu(conv4)

# Pooling Layer -> (?, 8, 8, 32)
pool4 = tf.nn.max_pool(conv4, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')



In [0]:
#########################
# ConvLayer 5, 6, 7 (원래 인풋: 56 * 56 * 128)

# for Conv Layer 05 filter / (3, 3, 128, 256) but (3, 3, 32, 64)
w5 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
b5 = tf.Variable(tf.random_normal([64], stddev=0.01))

# Convolution Layer -> (?, 8, 8, 64)
conv5 = tf.add(tf.nn.conv2d(pool4, w5, strides=[1, 1, 1, 1], padding='SAME'), b5)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv5, [0])
conv5 = tf.nn.batch_normalization(conv5, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv5 = tf.nn.relu(conv5)

# for Conv Layer 06 filter / (3, 3, 256, 256) but (3, 3, 64, 64)
w6 = tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=0.01))
b6 = tf.Variable(tf.random_normal([64], stddev=0.01))

# Convolution Layer 02 -> (?, 8, 8, 64)
conv6 = tf.add(tf.nn.conv2d(conv5, w6, strides=[1, 1, 1, 1], padding='SAME'), b6)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv6, [0])
conv6 = tf.nn.batch_normalization(conv6, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv6 = tf.nn.relu(conv6)

# for Conv Layer 07 filter / (3, 3, 256, 256) but (3, 3, 64, 64)
w7 = tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=0.01))
b7 = tf.Variable(tf.random_normal([64], stddev=0.01))

# Convolution Layer -> (?, 8, 8, 64)
conv7 = tf.add(tf.nn.conv2d(conv6, w7, strides=[1, 1, 1, 1], padding='SAME'), b7)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv7, [0])
conv7 = tf.nn.batch_normalization(conv7, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv7 = tf.nn.relu(conv7)

# Pooling Layer -> (?, 4, 4, 64)
pool7 = tf.nn.max_pool(conv7, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

##############################################################################################################################
# 32x32 CIFAR-10 이미지의 경우, 더 진행하면 max pooling으로 인해 차원이 너무 축소되어 버려 의미가 없음.

#########################
# ConvLayer 8, 9, 10 (원래 인풋: 28 * 28 * 256)

# # for Conv Layer 08 filter / (3, 3, 256, 512)
# w8 = tf.Variable(tf.random_normal([3, 3, 256, 512], stddev=0.01))
# b8 = tf.Variable(tf.random_normal([512], stddev=0.01))

# # Convolution Layer -> (?, 28, 28, 512)
# conv8 = tf.add(tf.nn.conv2d(pool7, w8, strides=[1, 1, 1, 1], padding='SAME'), b8)

# # relu
# conv8 = tf.nn.relu(conv8)

# # for Conv Layer 09 filter / (3, 3, 512, 512)
# w9 = tf.Variable(tf.random_normal([3, 3, 512, 512], stddev=0.01))
# b9 = tf.Variable(tf.random_normal([512], stddev=0.01))

# # Convolution Layer -> (?, 28, 28, 512)
# conv9 = tf.add(tf.nn.conv2d(conv8, w9, strides=[1, 1, 1, 1], padding='SAME'), b9)

# # relu
# conv9 = tf.nn.relu(conv9)

# # for Conv Layer 10 filter / (3, 3, 512, 512)
# w10 = tf.Variable(tf.random_normal([3, 3, 512, 512], stddev=0.01))
# b10 = tf.Variable(tf.random_normal([512], stddev=0.01))

# # Convolution Layer -> (?, 28, 28, 512)
# conv10 = tf.add(tf.nn.conv2d(conv9, w10, strides=[1, 1, 1, 1], padding='SAME'), b10)

# # relu
# conv10 = tf.nn.relu(conv10)

# # Pooling Layer -> (?, 14, 14, 512)
# pool10 = tf.nn.max_pool(conv10, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


# #########################
# # ConvLayer 11, 12, 13 (원래 인풋: 14 * 14 * 512)

# # for Conv Layer 11 filter / (3, 3, 512, 512)
# w11 = tf.Variable(tf.random_normal([3, 3, 512, 512], stddev=0.01))
# b11 = tf.Variable(tf.random_normal([512], stddev=0.01))

# # Convolution Layer -> (?, 14, 14, 512)
# conv11 = tf.add(tf.nn.conv2d(conv10, w11, strides=[1, 1, 1, 1], padding='SAME'), b11)

# # relu
# conv11 = tf.nn.relu(conv11)

# # for Conv Layer 12 filter / (3, 3, 512, 512)
# w12 = tf.Variable(tf.random_normal([3, 3, 512, 512], stddev=0.01))
# b12 = tf.Variable(tf.random_normal([512], stddev=0.01))

# # Convolution Layer -> (?, 14, 14, 512)
# conv12 = tf.add(tf.nn.conv2d(conv11, w12, strides=[1, 1, 1, 1], padding='SAME'), b12)

# # relu
# conv12 = tf.nn.relu(conv12)

# # for Conv Layer 13 filter / (3, 3, 512, 512)
# w13 = tf.Variable(tf.random_normal([3, 3, 512, 512], stddev=0.01))
# b13 = tf.Variable(tf.random_normal([512], stddev=0.01))

# # Convolution Layer -> (?, 14, 14, 512)
# conv13 = tf.add(tf.nn.conv2d(conv12, w13, strides=[1, 1, 1, 1], padding='SAME'), b13)

# # relu
# conv13 = tf.nn.relu(conv13)

# # Pooling Layer -> (?, 7, 7, 512)
# pool13 = tf.nn.max_pool(conv13, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
##################################################
# Flatten layer & Fully Connected Layer

###############################
# Flatten layer
dim = pool7.get_shape().as_list()
flat_dim = dim[1] * dim[2] * dim[3] # 4 * 4 * 64
flat = tf.reshape(pool7, [-1, flat_dim])

###############################
# Fully Connected Layer 01 

# for Final FC / (7 * 7 * 512, 4096) but (4 * 4 * 64, 256)
wfc1 = tf.Variable(tf.random_normal([flat_dim, 256], stddev=0.01))
bfc1 = tf.Variable(tf.random_normal([256]))

# fc layer
fc1 = tf.add(tf.matmul(flat, wfc1), bfc1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(fc1, [0])
fc1 = tf.nn.batch_normalization(fc1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
fc1 = tf.nn.relu(fc1)
# dropout
fc1 = tf.nn.dropout(fc1, kp)

###############################
# Fully Connected Layer 02

# for Final FC / (4096, 4096) but (256, 256)
wfc2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
bfc2 = tf.Variable(tf.random_normal([256]))

# fc layer
fc2 = tf.add(tf.matmul(fc1, wfc2), bfc2)

# batch normalization
batch_mean, batch_var = tf.nn.moments(fc2, [0])
fc2 = tf.nn.batch_normalization(fc2, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
fc2 = tf.nn.relu(fc2)
# dropout
fc2 = tf.nn.dropout(fc2, kp)

###############################
# Fully Connected Layer 03

# for Final FC / (4096, 1000) but (256, 10)
wfc3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
bfc3 = tf.Variable(tf.random_normal([10]))

# fc layer
logits = tf.add(tf.matmul(fc2, wfc3), bfc3)

#########################
# Cost & Optimizer
# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y), name="cost")
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
    
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [0]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')

epochs = 20
batch_size = 256
learning_rate = 0.01
dropout = 0.7
n_of_batches = int(len(train_x) / batch_size)

for epoch in range(epochs): 
    print("%dth epoch" % (epoch + 1))

    for i in range(n_of_batches):

        X_batch = train_x[(i * batch_size):((i + 1) * batch_size)]
        Y_batch = train_y[(i * batch_size):((i + 1) * batch_size)]

        sess.run(optimizer, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                       lr: learning_rate, kp: dropout})

        # 학습 상황 디스플레이
        if ((i + 1) % 10 == 0): 
            loss, accuracy = sess.run([cost, acc], 
                                      feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                                 lr: learning_rate, kp: 1.0})
            print("%dth records, training cost: %.3f, accuracy: %.2f" % ((i + 1) * batch_size, loss, accuracy * 100))

print("Training Complete")

In [0]:
###########################
# Evaluation
# accuracy = sess.run(acc, feed_dict={X: test_x, Y: np.eye(10)[test_y], lr: learning_rate, kp: 1.0})
# print('Accuracy: %.2f' % (accuracy * 100))

accuracy = 0.0
n_of_batches = int(len(test_x) / batch_size)
for i in range(n_of_batches):

    X_batch = test_x[(i * batch_size):((i + 1) * batch_size)]
    Y_batch = test_y[(i * batch_size):((i + 1) * batch_size)]

    accuracy += sess.run(acc, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], lr: learning_rate, kp: 1.0})
    
print('Accuracy: %.2f' % ((accuracy / n_of_batches) * 100))



## CIFAR-10 + GoogLeNet

http://joelouismarino.github.io/blog_posts/blog_googlenet_keras.html

![image](http://joelouismarino.github.io/images/blog_images/blog_googlenet_keras/googlenet_stem.png)
![image](http://joelouismarino.github.io/images/blog_images/blog_googlenet_keras/googlenet_auxiliary.png)
![image](http://joelouismarino.github.io/images/blog_images/blog_googlenet_keras/googlenet_output.png)
![image](https://cdn-images-1.medium.com/max/1600/1*66hY3zZTf0Lw2ItybiRxyg.png)


In [0]:
import tensorflow as tf
import numpy as np
import random

# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders
lr = tf.placeholder(tf.float32) # learning rate
kp = tf.placeholder(tf.float32) # dropout

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


In [0]:
#########################
# ConvLayer 1, 2

# for Conv Layer 01 filter
w1 = tf.Variable(tf.random_normal([3, 3, 3, 16], stddev=0.01))
b1 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 32, 32, 16)
conv1 = tf.add(tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME'), b1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv1, [0])
conv1 = tf.nn.batch_normalization(conv1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv1 = tf.nn.relu(conv1)

# LRN
# depth_radius=5, bias=1.0, alpha=1e-04, beta=0.75
norm1 = tf.nn.local_response_normalization(conv1, 5, 1.0, 1e-04, 0.75)

# for Conv Layer 02 filter
w2 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01))
b2 = tf.Variable(tf.random_normal([32], stddev=0.01))

# Convolution Layer -> (?, 32, 32, 32)
conv2 = tf.add(tf.nn.conv2d(norm1, w2, strides=[1, 1, 1, 1], padding='SAME'), b2)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv2, [0])
conv2 = tf.nn.batch_normalization(conv2, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv2 = tf.nn.relu(conv2)

# LRN
# depth_radius=5, bias=1.0, alpha=1e-04, beta=0.75
norm2 = tf.nn.local_response_normalization(conv2, 5, 1.0, 1e-04, 0.75)

# Pooling Layer -> (?, 16, 16, 32)
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:

#########################
# ConvLayer 3, 4

# for Conv Layer 03-1 filter
w31 = tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=0.01))
b31 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv31 = tf.add(tf.nn.conv2d(pool2, w31, strides=[1, 1, 1, 1], padding='SAME'), b31)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv31, [0])
conv31 = tf.nn.batch_normalization(conv31, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv31 = tf.nn.relu(conv31)

#-------------------------------
# for Conv Layer 03-2-1 filter
w321 = tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=0.01))
b321 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv321 = tf.add(tf.nn.conv2d(pool2, w321, strides=[1, 1, 1, 1], padding='SAME'), b321)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv321, [0])
conv321 = tf.nn.batch_normalization(conv321, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv321 = tf.nn.relu(conv321)

# for Conv Layer 03-2-2 filter
w322 = tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=0.01))
b322 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv322 = tf.add(tf.nn.conv2d(conv321, w322, strides=[1, 1, 1, 1], padding='SAME'), b322)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv322, [0])
conv32 = tf.nn.batch_normalization(conv322, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv32 = tf.nn.relu(conv32)

#-------------------------------
# for Conv Layer 03-3-1 filter
w331 = tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=0.01))
b331 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv331 = tf.add(tf.nn.conv2d(pool2, w331, strides=[1, 1, 1, 1], padding='SAME'), b331)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv331, [0])
conv331 = tf.nn.batch_normalization(conv331, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv331 = tf.nn.relu(conv331)

# for Conv Layer 03-3-2 filter
w332 = tf.Variable(tf.random_normal([5, 5, 8, 8], stddev=0.01))
b332 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer 02 -> (?, 16, 16, 8)
conv331 = tf.add(tf.nn.conv2d(conv331, w332, strides=[1, 1, 1, 1], padding='SAME'), b332)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv331, [0])
conv33 = tf.nn.batch_normalization(conv331, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv33 = tf.nn.relu(conv33)

#-------------------------------
# for Conv Layer 03-4-1 filter

# Pooling Layer -> (?, 16, 16, 32)
pool34 = tf.nn.max_pool(pool2, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')

# for Conv Layer 03-4 filter
w34 = tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=0.01))
b34 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv34 = tf.add(tf.nn.conv2d(pool34, w34, strides=[1, 1, 1, 1], padding='SAME'), b34)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv34, [0])
conv34 = tf.nn.batch_normalization(conv34, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv34 = tf.nn.relu(conv34)

# concat => (?, 16, 16, 32)
conv3 = tf.concat([conv31, conv32, conv33, conv34], 3)

# Pooling Layer -> (?, 8, 8, 32)
pool3 = tf.nn.avg_pool(conv3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
##################################################
# Flatten layer & Fully Connected Layer

###############################
# Flatten layer
dim = pool3.get_shape().as_list()
flat_dim = dim[1] * dim[2] * dim[3] # 8 * 8 * 32
flat = tf.reshape(pool3, [-1, flat_dim])

###############################
# Fully Connected Layer 01

# for Final FC
wfc1 = tf.Variable(tf.random_normal([flat_dim, 256], stddev=0.01))
bfc1 = tf.Variable(tf.random_normal([256]))

# fc layer
fc1 = tf.add(tf.matmul(flat, wfc1), bfc1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(fc1, [0])
fc1 = tf.nn.batch_normalization(fc1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
fc1 = tf.nn.relu(fc1)
# dropout
fc1 = tf.nn.dropout(fc1, kp)

###############################
# Fully Connected Layer 02

# for Final FC 
wfc2 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
bfc2 = tf.Variable(tf.random_normal([10]))

# fc layer
logits = tf.add(tf.matmul(fc1, wfc2), bfc2)

#########################
# Cost & Optimizer

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y), name="cost")
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
    
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [0]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')
epochs = 20
batch_size = 256
learning_rate = 0.01
dropout = 0.7
n_of_batches = int(len(train_x) / batch_size)

for epoch in range(epochs): 
    print("%dth epoch" % (epoch + 1))

    for i in range(n_of_batches):
        X_batch = train_x[(i * batch_size):((i + 1) * batch_size)]
        Y_batch = train_y[(i * batch_size):((i + 1) * batch_size)]

        sess.run(optimizer, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                       lr: learning_rate, kp: dropout})

        # 학습 상황 디스플레이
        if ((i + 1) % 10 == 0): 
            loss, accuracy = sess.run([cost, acc], 
                                      feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                                 lr: learning_rate, kp: 1.0})
            print("%dth records, training cost: %.3f, accuracy: %.2f" % ((i + 1) * batch_size, loss, accuracy * 100))

print("Training Complete")

In [0]:
###########################
# Evaluation
# accuracy = sess.run(acc, feed_dict={X: test_x, Y: np.eye(10)[test_y], lr: learning_rate, kp: 1.0})
# print('Accuracy: %.2f' % (accuracy * 100))

accuracy = 0.0
n_of_batches = int(len(test_x) / batch_size)
for i in range(n_of_batches):

    X_batch = test_x[(i * batch_size):((i + 1) * batch_size)]
    Y_batch = test_y[(i * batch_size):((i + 1) * batch_size)]

    accuracy += sess.run(acc, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], lr: learning_rate, kp: 1.0})
    
print('Accuracy: %.2f' % ((accuracy / n_of_batches) * 100))



## CIFAR-10 + ResNet

![image](https://cdn-images-1.medium.com/max/1314/1*S3TlG0XpQZSIpoDIUCQ0RQ.jpeg)

![image](https://cdn-images-1.medium.com/max/1600/1*aq0q7gCvuNUqnMHh4cpnIw.png)

![image](https://1.bp.blogspot.com/-J5pbMKnnmZ0/V9t_aeGiEnI/AAAAAAAAB6c/f8-B42TxWAwasaXYpzKyOzUT2DqgpfZ9QCEw/s1600/%25E1%2584%2591%25E1%2585%25A1%25E1%2584%258B%25E1%2585%25B5%25E1%2586%25AF%2B2016.%2B9.%2B16.%2B%25E1%2584%258B%25E1%2585%25A9%25E1%2584%258C%25E1%2585%25A5%25E1%2586%25AB%2B9%2B10%2B32.jpeg)

https://m.blog.naver.com/PostView.nhn?blogId=laonple&logNo=220793640991&proxyReferer=https%3A%2F%2Fwww.google.co.kr%2F


In [0]:
import tensorflow as tf
import numpy as np
import random

# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders
lr = tf.placeholder(tf.float32) # learning rate
kp = tf.placeholder(tf.float32) # dropout

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


In [0]:
#########################
# ConvLayer 01

# for Conv Layer 01 filter
w1 = tf.Variable(tf.random_normal([5, 5, 3, 16], stddev=0.01))
b1 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 32, 32, 16)
conv1 = tf.add(tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME'), b1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv1, [0])
conv1 = tf.nn.batch_normalization(conv1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv1 = tf.nn.relu(conv1)

# Pooling Layer 01 -> (?, 16, 16, 16)
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
#########################
# ConvLayer 02

# for Conv Layer 02-1 filter
w21 = tf.Variable(tf.random_normal([1, 1, 16, 8], stddev=0.01))
b21 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv21 = tf.add(tf.nn.conv2d(pool1, w21, strides=[1, 1, 1, 1], padding='SAME'), b21)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv21, [0])
conv21 = tf.nn.batch_normalization(conv21, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv21 = tf.nn.relu(conv21)

# for Conv Layer 02-2 filter
w22 = tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=0.01))
b22 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv22 = tf.add(tf.nn.conv2d(conv21, w22, strides=[1, 1, 1, 1], padding='SAME'), b22)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv22, [0])
conv22 = tf.nn.batch_normalization(conv22, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv22 = tf.nn.relu(conv22)

# for Conv Layer 02-3 filter
w23 = tf.Variable(tf.random_normal([1, 1, 8, 16], stddev=0.01))
b23 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 16)
conv23 = tf.add(tf.nn.conv2d(conv22, w23, strides=[1, 1, 1, 1], padding='SAME'), b23)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv23, [0])
conv23 = tf.nn.batch_normalization(conv23, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# merge with pool1
conv2 = tf.add(conv23, pool1)

# relu -> (?, 16, 16, 16)
conv2 = tf.nn.relu(conv2)


In [0]:
#########################
# ConvLayer 03

# for Conv Layer 03-1 filter
w31 = tf.Variable(tf.random_normal([1, 1, 16, 8], stddev=0.01))
b31 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv31 = tf.add(tf.nn.conv2d(conv2, w31, strides=[1, 1, 1, 1], padding='SAME'), b31)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv31, [0])
conv31 = tf.nn.batch_normalization(conv31, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv31 = tf.nn.relu(conv31)

# for Conv Layer 03-2 filter
w32 = tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=0.01))
b32 = tf.Variable(tf.random_normal([8], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 8)
conv32 = tf.add(tf.nn.conv2d(conv31, w32, strides=[1, 1, 1, 1], padding='SAME'), b32)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv32, [0])
conv32 = tf.nn.batch_normalization(conv32, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv32 = tf.nn.relu(conv32)

# for Conv Layer 03-3 filter
w33 = tf.Variable(tf.random_normal([1, 1, 8, 16], stddev=0.01))
b33 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 16)
conv33 = tf.add(tf.nn.conv2d(conv32, w33, strides=[1, 1, 1, 1], padding='SAME'), b33)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv33, [0])
conv33 = tf.nn.batch_normalization(conv33, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# merge with pool1
conv3 = tf.add(conv33, conv2)

# relu -> (?, 16, 16, 16)
conv3 = tf.nn.relu(conv3)


In [0]:
#########################
# ConvLayer 04

# for Conv Layer 04 filter
w41 = tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=0.01))
b41 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 16)
conv41 = tf.add(tf.nn.conv2d(conv3, w41, strides=[1, 1, 1, 1], padding='SAME'), b41)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv41, [0])
conv41 = tf.nn.batch_normalization(conv41, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv41 = tf.nn.relu(conv41)

# for Conv Layer 04 filter
w42 = tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=0.01))
b42 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 16)
conv42 = tf.add(tf.nn.conv2d(conv41, w42, strides=[1, 1, 1, 1], padding='SAME'), b42)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv42, [0])
conv42 = tf.nn.batch_normalization(conv42, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

conv4 = tf.add(conv42, conv3)

# relu
conv4 = tf.nn.relu(conv4)

In [0]:
#########################
# ConvLayer 05

# for Conv Layer 05 filter
w51 = tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=0.01))
b51 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 16)
conv51 = tf.add(tf.nn.conv2d(conv4, w51, strides=[1, 1, 1, 1], padding='SAME'), b51)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv51, [0])
conv51 = tf.nn.batch_normalization(conv51, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
conv51 = tf.nn.relu(conv51)

# for Conv Layer 04 filter
w52 = tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=0.01))
b52 = tf.Variable(tf.random_normal([16], stddev=0.01))

# Convolution Layer -> (?, 16, 16, 16)
conv52 = tf.add(tf.nn.conv2d(conv51, w52, strides=[1, 1, 1, 1], padding='SAME'), b52)

# batch normalization
batch_mean, batch_var = tf.nn.moments(conv52, [0])
conv52 = tf.nn.batch_normalization(conv52, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

conv5 = tf.add(conv52, conv4)

# relu
conv5 = tf.nn.relu(conv5)

# Pooling Layer -> (?, 8, 8, 16)
pool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
##################################################
# Flatten layer & Fully Connected Layer

###############################
# Flatten layer
dim = pool5.get_shape().as_list()
flat_dim = dim[1] * dim[2] * dim[3] # 8 * 8 * 16
flat = tf.reshape(pool5, [-1, flat_dim])

###############################
# Fully Connected Layer 01

# for Final FC
wfc1 = tf.Variable(tf.random_normal([flat_dim, 256], stddev=0.01))
bfc1 = tf.Variable(tf.random_normal([256]))

# fc layer
fc1 = tf.add(tf.matmul(flat, wfc1), bfc1)

# batch normalization
batch_mean, batch_var = tf.nn.moments(fc1, [0])
fc1 = tf.nn.batch_normalization(fc1, batch_mean, batch_var, None, None, 0.001) # epsilon = 0.001

# relu
fc1 = tf.nn.relu(fc1)
# dropout
fc1 = tf.nn.dropout(fc1, kp)

###############################
# Fully Connected Layer 02

# for Final FC
wfc2 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
bfc2 = tf.Variable(tf.random_normal([10]))

# fc layer
logits = tf.add(tf.matmul(fc1, wfc2), bfc2)

#########################
# Cost & Optimizer

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y), name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
    
with tf.name_scope("Prediction"):
    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")


In [0]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

print('Learning started. It takes sometime.')

epochs = 20
batch_size = 256
learning_rate = 0.01
dropout = 0.7
n_of_batches = int(len(train_x) / batch_size)

for epoch in range(epochs): 
    print("%dth epoch" % (epoch + 1))

    for i in range(n_of_batches):

        X_batch = train_x[(i * batch_size):((i + 1) * batch_size)]
        Y_batch = train_y[(i * batch_size):((i + 1) * batch_size)]

        sess.run(optimizer, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                       lr: learning_rate, kp: dropout})

        # 학습 상황 디스플레이
        if ((i + 1) % 10 == 0): 
            loss, accuracy = sess.run([cost, acc], 
                                      feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], 
                                                 lr: learning_rate, kp: 1.0})
            print("%dth records, training cost: %.3f, accuracy: %.2f" % ((i + 1) * batch_size, loss, accuracy * 100))

print("Training Complete")

In [0]:
###########################
# Evaluation
# accuracy = sess.run(acc, feed_dict={X: test_x, Y: np.eye(10)[test_y], lr: learning_rate, kp: 1.0})
# print('Accuracy: %.2f' % (accuracy * 100))

accuracy = 0.0
n_of_batches = int(len(test_x) / batch_size)
for i in range(n_of_batches):

    X_batch = test_x[(i * batch_size):((i + 1) * batch_size)]
    Y_batch = test_y[(i * batch_size):((i + 1) * batch_size)]

    accuracy += sess.run(acc, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch], lr: learning_rate, kp: 1.0})
    
print('Accuracy: %.2f' % ((accuracy / n_of_batches) * 100))

